# Stratégie pour mettre à jour le clustering au fur et à mesure des nouvelles transactions

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import datetime
import matplotlib.pyplot as plt

La stratégie va être de développer une fonction qui prends en entrée l'ensemble des données (anciennes + nouvelles) et qui calculetoutes les features (TotalInvoices, TotalQuantity, TotalPrice, Recency, CancellationPercentage, AvgItemsPerOrder)

In [33]:
df_ = pd.read_csv('dataset/clean_online_retail.csv')
df_all = df_.copy()

df_ = pd.read_csv('dataset/rfm_2009-2010.csv', index_col="Customer ID")
df_rfm_spe_2009_2010 = df_.copy()

df_ = pd.read_csv('dataset/rfm.csv', index_col="Customer ID")
df_rfm_spe_all = df_.copy()

df_2011 = df_all[df_all['InvoiceDate'].dt.year >=2011]
df_2009_2010 = df_all[df_all['InvoiceDate'].dt.year <= 2010]

df_all["InvoiceDate"] = pd.to_datetime(df_all["InvoiceDate"])

In [60]:
import pandas as pd

def calculate_rfm(df):
    last_date = df["InvoiceDate"].max()
    recency = (last_date - df.groupby("Customer ID")["InvoiceDate"].max()).dt.days

    df_gp_cancellation = df.groupby("Customer ID").agg({"cancelled": "sum", "Invoice": "count"})
    df_gp_cancellation = df_gp_cancellation.rename(columns={"cancelled": "TotalCancelled", "Invoice": "TotalInvoices"})
    df_gp_cancellation["CancellationPercentage"] = (df_gp_cancellation["TotalCancelled"] / df_gp_cancellation["TotalInvoices"]) * 100

    df_gp_nb_articles_mean = df.groupby(['Customer ID', 'Invoice']).agg({'Quantity': 'sum'})
    df_gp_nb_articles_mean = df_gp_nb_articles_mean.groupby('Customer ID').mean()
    df_gp_nb_articles_mean.columns = ['AvgItemsPerOrder']

    df_rfm_spe = df.groupby('Customer ID').agg({'Invoice': 'nunique', 'Quantity': 'sum', 'TotalPrice': 'sum'})
    df_rfm_spe.columns = ['TotalInvoices', 'TotalQuantity', 'TotalPrice']
    df_rfm_spe["Recency"] = recency
    df_rfm_spe = df_rfm_spe.merge(df_gp_cancellation["CancellationPercentage"], on="Customer ID")
    df_rfm_spe = df_rfm_spe.merge(df_gp_nb_articles_mean["AvgItemsPerOrder"], on="Customer ID")

    return df_rfm_spe


In [61]:
df = pd.concat([df_2009_2010, df_2011])
df_rfm_spe_updated = calculate_rfm(df)


Pour vérifier que la fonction fonctionne correctement, voici une comparaison des résultats

d'abord les résultats obtenus avec la fonction

In [62]:
df_rfm_spe_updated.describe()

,TotalInvoices,TotalQuantity,TotalPrice,Recency,CancellationPercentage,AvgItemsPerOrder
count,5939.000000,5939.000000,5939.000000,5939.000000,5939.000000,5939.000000
mean,7.555144,1690.684627,2742.884541,201.784812,3.688378,200.185120
std,15.970199,8480.714144,13679.955199,211.727459,11.943487,1221.908065
min,1.000000,-1085.000000,-25111.090000,0.000000,0.000000,-393.000000
25%,2.000000,177.000000,321.365000,24.000000,0.000000,74.875000
50%,4.000000,462.000000,823.530000,95.000000,0.000000,130.000000
75%,8.000000,1309.500000,2143.280000,380.000000,2.647082,215.391667
max,510.000000,364580.000000,570380.610000,738.000000,100.000000,87167.000000


maintenant les résultats obtenus précédements dans le notebook ou les features sont calculés pour l'ensemble du dataset

In [63]:
df_rfm_spe_all.describe()

,TotalInvoices,TotalQuantity,TotalPrice,Recency,CancellationPercentage,AvgItemsPerOrder
count,5939.000000,5939.000000,5939.000000,5939.000000,5939.000000,5939.000000
mean,7.555144,1690.684627,2742.884541,201.784812,3.688378,200.185120
std,15.970199,8480.714144,13679.955199,211.727459,11.943487,1221.908065
min,1.000000,-1085.000000,-25111.090000,0.000000,0.000000,-393.000000
25%,2.000000,177.000000,321.365000,24.000000,0.000000,74.875000
50%,4.000000,462.000000,823.530000,95.000000,0.000000,130.000000
75%,8.000000,1309.500000,2143.280000,380.000000,2.647082,215.391667
max,510.000000,364580.000000,570380.610000,738.000000,100.000000,87167.000000


Les statistiques sont identiques, on a bien obtenus le même résultats.

On peut ensuite imaginer d'autres fonctions :
- la normalisation des données
- la méthode du coude
- le clustering avec le nombre de coude trouvé
- PCA
- la supression des lignes dans les clusters de moins de x clients (ces clients seront à traité ultérierement, dans ce clustering on les considèrera comme des valeurs abérrantes, on notera leur id pour de futures analyses)
- nouveau clustering avec moins de cluster
- PCA
- analyse statistique des cluster

Toutes ces fonctions pourront être utilisées dans un script lancé régulièrement pour mettre à jours la segmentation